# Managing population growth

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

Much of the Population growth is handled in scripts inside the Azure Function

In [2]:
# mapping to the modules that make the app

sys.path.insert(0, "../..")
sys.path.insert(0, "..")
sys.path.insert(0, "../../functions/popgrowth/")

from app.connectors import cmdb_graph
from app.objects import time
from helpers import test_queries

c = cmdb_graph.CosmosdbClient()



executing local windows deployment


In [6]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"app/configurations/popgrowthconfig.yaml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [7]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at: 2023-09-04T18:32:22.606175+00:00 UTU:28340 >

In [8]:
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 28340}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

## Population Resource Consumption

In [10]:
from app.functions import consumption

We're going to run a resourse query test to ensure that consumption is happening. To test, update the `objid` with a planet that has consuming pops. 

In [12]:
validate_resources_updated_query = f"""
g.V().has('objid','0759161021157').out('has').valueMap()
"""
print(validate_resources_updated_query)
c.run_query(validate_resources_updated_query)
pre_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))
pre_consumption


g.V().has('objid','0759161021157').out('has').valueMap()



,name,objid,volume,max_volume,description,replenish_rate,username,objtype,id
0,Organic,8490375029898,1023,1023,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8490375029898
1,Common Minerals,5278459986296,101,101,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,5278459986296
2,Rare Minerals,7509025449050,37,37,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,7509025449050
3,Water,0191279521870,9218,9218,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,0191279521870


In [13]:

all_pops_query = f"""
g.V().has('label','pop').as('pop')
    .local(
        union(
            out('inhabits').as('location'),
            out('isOf').as('species')
            )
            .fold()).as('location','species')
        .path()
        .by(unfold().valueMap().fold())
"""

In [15]:
c.run_query(all_pops_query)
data = c.reduce_res(c.res)

had an issue with , 2 species [{'name': 'Gueky Van', 'objid': '8421970458164', 'conformity': 0.364, 'literacy': 0.513, 'aggression': 0.433, 'constitution': 0.326, 'health': 0.7, 'isIn': '0588407234794', 'industry': 0.3795, 'wealth': 0.4462, 'factionLoyalty': 0.118, 'isIdle': 'true', 'username': 'BillmanLocal2', 'objtype': 'pop', 'id': '8421970458164'}, {'name': 'Foris', 'class': 'terrestrial', 'objid': '0759161021157', 'radius': 0.586, 'mass': 1.621, 'orbitsDistance': 0.575, 'orbitsId': '7157405905541', 'orbitsName': 'Jar', 'isSupportsLife': 'true', 'isPopulated': 'true', 'isHomeworld': 'true', 'username': 'BillmanLocal2', 'objtype': 'planet', 'id': '0759161021157'}]
had an issue with , 2 species [{'name': 'Gueky Ri', 'objid': '2697245290546', 'conformity': 0.407, 'literacy': 0.534, 'aggression': 0.391, 'constitution': 0.322, 'health': 0.7, 'isIn': '0588407234794', 'industry': 0.3565, 'wealth': 0.4452, 'factionLoyalty': 0.49, 'isIdle': 'true', 'username': 'BillmanLocal2', 'objtype': 'p

In [18]:
import operator
from functools import reduce

def clean_node(x):
    # For each value, return the last value in the array for that object. 
    for k in list(x.keys()):
        if type(x[k])==list:
            x[k] = x[k][-1]
    if 'objid' in x.keys():
        x["id"] = x["objid"]
    return x

def clean_nodes(self, nodes):
    return [self.clean_node(n) for n in nodes]

def reduce_res(res):
    fab = []
    for n,r in enumerate(res): 
        t = {}
        labels = reduce(operator.concat, r['labels'])
        objects = reduce(operator.concat, r['objects'])

        for i,l in enumerate(labels):
            try:
                t[l]=clean_node(objects[i])
            except:
                print("had an issue with ,",i,l, objects)
        fab.append(t)
    return fab

reduce_res(c.res)


TypeError: reduce_res() missing 1 required positional argument: 'res'

In [14]:
c.run_query(all_pops_query)
data = c.reduce_res(c.res)
pd.DataFrame(data)

had an issue with , 2 species [{'name': 'Gueky Van', 'objid': '8421970458164', 'conformity': 0.364, 'literacy': 0.513, 'aggression': 0.433, 'constitution': 0.326, 'health': 0.7, 'isIn': '0588407234794', 'industry': 0.3795, 'wealth': 0.4462, 'factionLoyalty': 0.118, 'isIdle': 'true', 'username': 'BillmanLocal2', 'objtype': 'pop', 'id': '8421970458164'}, {'name': 'Foris', 'class': 'terrestrial', 'objid': '0759161021157', 'radius': 0.586, 'mass': 1.621, 'orbitsDistance': 0.575, 'orbitsId': '7157405905541', 'orbitsName': 'Jar', 'isSupportsLife': 'true', 'isPopulated': 'true', 'isHomeworld': 'true', 'username': 'BillmanLocal2', 'objtype': 'planet', 'id': '0759161021157'}]
had an issue with , 2 species [{'name': 'Gueky Ri', 'objid': '2697245290546', 'conformity': 0.407, 'literacy': 0.534, 'aggression': 0.391, 'constitution': 0.322, 'health': 0.7, 'isIn': '0588407234794', 'industry': 0.3565, 'wealth': 0.4452, 'factionLoyalty': 0.49, 'isIdle': 'true', 'username': 'BillmanLocal2', 'objtype': 'p

,pop,location,species
0,"{'health': -0.04999999999999993, 'name': 'Tilc...","{'name': 'Penbufergui', 'class': 'terrestrial'...","{'name': 'Towner', 'objid': '1245644824657', '..."
1,"{'health': -0.04999999999999993, 'name': 'Tilc...","{'name': 'Penbufergui', 'class': 'terrestrial'...","{'name': 'Towner', 'objid': '1245644824657', '..."
2,"{'health': -0.04999999999999993, 'name': 'Tilc...","{'name': 'Penbufergui', 'class': 'terrestrial'...","{'name': 'Towner', 'objid': '1245644824657', '..."
3,"{'health': -0.04999999999999993, 'name': 'Tilc...","{'name': 'Penbufergui', 'class': 'terrestrial'...","{'name': 'Towner', 'objid': '1245644824657', '..."
4,"{'health': -0.04999999999999993, 'name': 'Tilc...","{'name': 'Penbufergui', 'class': 'terrestrial'...","{'name': 'Towner', 'objid': '1245644824657', '..."
...,...,...,...
226,"{'name': 'Gueky Piahy', 'objid': '478587317643...","{'name': 'Foris', 'class': 'terrestrial', 'obj...",NaN
227,"{'name': 'Springstal Richda', 'objid': '505050...","{'name': 'Foris', 'class': 'terrestrial', 'obj...",NaN
228,"{'name': 'Gueky Sandenzomay', 'objid': '697847...","{'name': 'Foris', 'class': 'terrestrial', 'obj...",NaN
229,"{'name': 'Springstal Oszu', 'objid': '81673373...","{'name': 'Foris', 'class': 'terrestrial', 'obj...",NaN


In [10]:
pops_df = pd.DataFrame([d['pop'] for d in data])
pops_df

,health,isIdle,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,username,objtype,id
0,0.15,false,Obattla Magor,7467985401518,0.421,0.376,0.632,0.338,8822366435138,0.4850,0.4305,0.505,BillmanLocal2,pop,7467985401518
1,0.15,true,Obattla Pe,5100638967625,0.325,0.352,0.606,0.450,8822366435138,0.5280,0.4400,0.459,BillmanLocal2,pop,5100638967625
2,0.20,true,Obattla Van,1800951322768,0.446,0.390,0.549,0.573,8822366435138,0.5610,0.4755,0.423,BillmanLocal2,pop,1800951322768
3,0.20,true,Obattla Rochi,0692394121482,0.452,0.395,0.514,0.546,8822366435138,0.5300,0.4625,0.354,BillmanLocal2,pop,0692394121482
4,0.20,true,Hassia Sin,0248671359952,0.623,0.215,0.704,0.569,4146242421204,0.6365,0.4258,0.662,BillmanLocal2,pop,0248671359952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,0.70,true,Aythakorei Burghuatoucal,2128380883047,0.473,0.418,0.547,0.445,3387227570408,0.4960,0.4570,0.472,Billmanserver,pop,2128380883047
209,0.70,true,Aythakorei Ton,6992584860189,0.429,0.447,0.467,0.416,3387227570408,0.4415,0.4442,0.252,Billmanserver,pop,6992584860189
210,0.70,true,Borgantasga Nak,7444602298651,0.766,0.437,0.356,0.499,6740440850433,0.4275,0.4322,0.806,Billmanserver,pop,7444602298651
211,0.70,true,Borgantasga Bin,7852618490329,0.569,0.357,0.409,0.640,6740440850433,0.5245,0.4408,0.501,Billmanserver,pop,7852618490329


In [11]:
species_df = pd.DataFrame([d['species'] for d in data])
species_df

,name,objid,consumes,effuses,viral_resilience,habitat_resilience,username,objtype,id
0,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
1,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
2,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
3,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
4,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
...,...,...,...,...,...,...,...,...,...
208,Hor,6688039622667,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,6688039622667
209,Hor,6688039622667,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,6688039622667
210,Hor,6688039622667,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,6688039622667
211,Hor,6688039622667,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,6688039622667


In [12]:
locations_df = pd.DataFrame([d['location'] for d in data])
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,username,objtype,id
0,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
1,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
2,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
3,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
4,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Nacor,terrestrial,1425359096973,1.050,0.000,1.083,8319410016906,Hi,true,true,true,Billmanserver,planet,1425359096973
209,Nacor,terrestrial,1425359096973,1.050,0.000,1.083,8319410016906,Hi,true,true,true,Billmanserver,planet,1425359096973
210,Nacor,terrestrial,1425359096973,1.050,0.000,1.083,8319410016906,Hi,true,true,true,Billmanserver,planet,1425359096973
211,Nacor,terrestrial,1425359096973,1.050,0.000,1.083,8319410016906,Hi,true,true,true,Billmanserver,planet,1425359096973


First getting the list of consumption for each pop.


In [13]:
pops_df,species_df,locations_df = consumption.all_pops_consumption(c)


We expand that dataset to break open the list of people in each pop. 

In [14]:
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 28316}

In [15]:
consumption_df = consumption.get_consumption_df(locations_df,species_df,params)
consumption_df = consumption.expand_consumption_df(consumption_df)
consumption_df

c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ser.consumes = j
c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ser.consumes = j
c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

,location_id,consumes,pop,consumption,multi
0,1425359096973,Organic,7,14,True
1,2816844503430,Organic,84,168,True
2,2972908340606,Organic,15,30,True
3,4734834255576,Organic,9,18,True
4,5640544774717,Organic,98,196,True



Then we get a list of the resources available on that location. 

In [16]:
c.run_query(consumption.make_resource_query(consumption_df))
resources = c.res
resources

[{'labels': [['location'], ['resource']],
  'objects': [{'objid': ['2972908340606'], 'name': ['Dorf']},
   {'volume': [-1], 'objid': ['8679787925290'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['5640544774717'], 'name': ['Luraszi']},
   {'volume': [-1], 'objid': ['4182507921316'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['2816844503430'], 'name': ['Penbufergui']},
   {'volume': [-1], 'objid': ['6331044458818'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['4734834255576'], 'name': ['Vil']},
   {'volume': [1042], 'objid': ['6733191593029'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['1425359096973'], 'name': ['Nacor']},
   {'volume': [996], 'objid': ['5721907953807'], 'name': ['Organic']}]}]

In [17]:
consumption_df = consumption.tally_consumption(c,consumption_df,resources)
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,1425359096973,Organic,7,14,True,996.0,982.0
1,2816844503430,Organic,84,168,True,-1.0,-1.0
2,2972908340606,Organic,15,30,True,-1.0,-1.0
3,4734834255576,Organic,9,18,True,1042.0,1024.0
4,5640544774717,Organic,98,196,True,-1.0,-1.0


Then we update the resources by taking out the resources that exist. You can check that this has happened in the graph by looking at the location ids. This is a verification query that isn't run in the function application.

In [18]:
consumption_df.apply(lambda x: consumption.make_resource_update_query(c,x),axis=1)

0    None
1    None
2    None
3    None
4    None
dtype: object

In [19]:
c.run_query(validate_resources_updated_query)
post_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))

In [20]:
pre_consumption

,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,-1,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


Only the resources that are consumed will diminish, and they will stop diminishing if the level goes below `0`

In [21]:
post_consumption

,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,-1,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


Now for the starving. Populations without enough food will starve (health declines). Populations with < 0 health will die.

In [22]:
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,1425359096973,Organic,7,14,True,996.0,982.0
1,2816844503430,Organic,84,168,True,-1.0,-1.0
2,2972908340606,Organic,15,30,True,-1.0,-1.0
3,4734834255576,Organic,9,18,True,1042.0,1024.0
4,5640544774717,Organic,98,196,True,-1.0,-1.0


In [23]:
consumption_df[consumption_df['remaining']<=0].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

84 pops will starve in 2816844503430
15 pops will starve in 2972908340606
98 pops will starve in 5640544774717


1    None
2    None
4    None
dtype: object

Looking more closely at `lower_health`
This only happens on locations that have < 0 resources. But here we're going to do it regardless

In [24]:
x = consumption_df.loc[0]
print(x)

dead_pop_nodes = []
dead_pop_ids = []
death_event_edges = []
query =f"""
g.V().has('objid','{x.location_id}').as('location').in('inhabits')
    .haslabel('pop').as('pop')
    .out('isOf').as('species')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('name','objid','health','username'))
        .by(valueMap('name','objid','consumes'))
"""
c.run_query(query)
out = c.res
out[0]

location_id    1425359096973
consumes             Organic
pop                        7
consumption               14
multi                   True
available              996.0
remaining              982.0
Name: 0, dtype: object


{'labels': [['location'], ['pop'], ['species']],
 'objects': [{'objid': ['1425359096973'], 'name': ['Nacor']},
  {'name': ['Aythakorei Or'],
   'objid': ['9427021644478'],
   'health': [0.7],
   'username': ['Billmanserver']},
  {'name': ['Hor'], 'objid': ['6688039622667'], 'consumes': ['[Organic]']}]}

## Looking more closely at starvation

* Grab my testing planet, with people on it
* Lower that amount of resource on that planet to 0
* run the lower health metric
* watch the people starve and die

In [25]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').valuemap()
    """)

planet = c.clean_node(c.res[0])
planet

{'name': 'Nacor',
 'class': 'terrestrial',
 'objid': '1425359096973',
 'radius': 1.05,
 'mass': 0,
 'orbitsDistance': 1.083,
 'orbitsId': '8319410016906',
 'orbitsName': 'Hi',
 'isSupportsLife': 'true',
 'isPopulated': 'true',
 'isHomeworld': 'true',
 'username': 'Billmanserver',
 'objtype': 'planet',
 'id': '1425359096973'}

Grabbing that planets organics as well, and setting the amount of the organics to zero

In [26]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').out('has').has('name','Organic').property('volume',-1)
    """)

In [27]:
consumption_df.loc[consumption_df['location_id']==planet['objid'],['remaining','available']] = -1
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,1425359096973,Organic,7,14,True,-1.0,-1.0
1,2816844503430,Organic,84,168,True,-1.0,-1.0
2,2972908340606,Organic,15,30,True,-1.0,-1.0
3,4734834255576,Organic,9,18,True,1042.0,1024.0
4,5640544774717,Organic,98,196,True,-1.0,-1.0


In [28]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').out('has').valuemap()
    """)

organics = c.clean_node(c.res[0])
organics

{'volume': -1,
 'name': 'Organic',
 'objid': '5721907953807',
 'max_volume': 996,
 'description': 'bilogical material that can be consumed by pops',
 'replenish_rate': 10,
 'username': 'Billmanserver',
 'objtype': 'resource',
 'id': '5721907953807'}

Now that the resources are gone, the population should start to starve. 

In [29]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

7 pops will starve in 1425359096973


0    None
dtype: object

In [30]:
health_query =f"""
    g.V().has('objid','{planet['objid']}').as('location').in('inhabits')
        .haslabel('pop').as('pop')
        .out('isOf').as('species')
        .path()
            .by(valueMap('objid','name'))
            .by(valueMap('name','objid','health','username'))
            .by(valueMap('name','objid','consumes'))
"""

In [31]:
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

,name,objid,health,username
0,[Aythakorei Or],[9427021644478],[0.6499999999999999],[Billmanserver]
1,[Aythakorei Rezche],[8742217486310],[0.6499999999999999],[Billmanserver]
2,[Aythakorei Burghuatoucal],[2128380883047],[0.6499999999999999],[Billmanserver]
3,[Aythakorei Ton],[6992584860189],[0.6499999999999999],[Billmanserver]
4,[Borgantasga Nak],[7444602298651],[0.6499999999999999],[Billmanserver]
5,[Borgantasga Bin],[7852618490329],[0.6499999999999999],[Billmanserver]
6,[Aythakorei Ziburgdiadan],[3415888692185],[0.6499999999999999],[Billmanserver]


Run the following cells again and again to watch the health go down. 

In [32]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

7 pops will starve in 1425359096973


0    None
dtype: object

In [33]:
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

,name,objid,health,username
0,[Aythakorei Or],[9427021644478],[0.5999999999999999],[Billmanserver]
1,[Aythakorei Rezche],[8742217486310],[0.5999999999999999],[Billmanserver]
2,[Aythakorei Burghuatoucal],[2128380883047],[0.5999999999999999],[Billmanserver]
3,[Aythakorei Ton],[6992584860189],[0.5999999999999999],[Billmanserver]
4,[Borgantasga Nak],[7444602298651],[0.5999999999999999],[Billmanserver]
5,[Borgantasga Bin],[7852618490329],[0.5999999999999999],[Billmanserver]
6,[Aythakorei Ziburgdiadan],[3415888692185],[0.5999999999999999],[Billmanserver]


In [46]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

7 pops will starve in 1425359096973


""
